 Teste de Integração do Pipeline PySpark
Este notebook executa um teste de integração do pipeline completo, desde a leitura do CSV até a geração da tabela gold.

In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
#importa o code_elevate_app package mesmo que esteja na pasta tests

In [ ]:
from pyspark.sql import SparkSession
from code_elevate_app.utils.reader import Reader
from code_elevate_app.utils.processing import Transformer
from code_elevate_app.utils.aggregator import Aggregator
import os

BRONZE_PATH = os.path.join("..", "table_destinations", "info_transportes.parquet")
SILVER_PATH = os.path.join("..", "table_destinations", "info_transportes_silver.parquet")
GOLD_PATH = os.path.join("..", "table_destinations", "info_corridas_do_dia.parquet")
CSV_PATH = os.path.join("..", "info_transportes.csv") 

spark = SparkSession.builder.appName("integration-test").getOrCreate()
reader = Reader(spark, CSV_PATH)
processing = Transformer(spark)
aggregator = Aggregator(spark)
print("Iniciando teste de integração...")

 Bronze: Leitura e persistência do CSV

In [ ]:
df_transportes = reader.csv_reader()
reader.csv_reader()
assert os.path.exists(BRONZE_PATH), "Arquivo bronze não foi criado."
print("Tudo certo até aqui, arquivo bronze criado.")

 Silver: Transformação e persistência

In [ ]:
df_bronze = processing.read_bronze(BRONZE_PATH)
df_silver = processing.transform(df_bronze)
processing.write_silver(df_silver, SILVER_PATH)
assert os.path.exists(SILVER_PATH), "Arquivo silver não foi criado."
print("Tudo certo até aqui, arquivo silver criado.")

 Gold: Agregação e persistência

In [ ]:
df_silver = aggregator.read_silver(SILVER_PATH)
df_gold = aggregator.aggregations(df_silver)
aggregator.write_gold(df_gold, GOLD_PATH)
assert os.path.exists(GOLD_PATH), "Arquivo gold não foi criado."
print("Tudo certo até aqui, arquivo gold criado.")

 Visualização dos resultados finais (gold)

In [ ]:
df_gold = spark.read.parquet(GOLD_PATH)
df_gold.show()

In [ ]:
spark.stop()